In [1]:
import numpy as np # linear algebra

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import os
%matplotlib inline
from subprocess import check_output


In [2]:
df = pd.read_csv("cdc_zika.csv",parse_dates=['report_date'],infer_datetime_format=True,index_col=0)
df.head(5)

C:\Users\RAVIT\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,location,location_type,data_field,data_field_code,time_period,time_period_type,value,unit
report_date,,,,,,,,
2016-03-19,Argentina-Buenos_Aires,province,cumulative_confirmed_local_cases,AR0001,NaN,NaN,0,cases
2016-03-19,Argentina-Buenos_Aires,province,cumulative_probable_local_cases,AR0002,NaN,NaN,0,cases
2016-03-19,Argentina-Buenos_Aires,province,cumulative_confirmed_imported_cases,AR0003,NaN,NaN,2,cases
2016-03-19,Argentina-Buenos_Aires,province,cumulative_probable_imported_cases,AR0004,NaN,NaN,1,cases
2016-03-19,Argentina-Buenos_Aires,province,cumulative_cases_under_study,AR0005,NaN,NaN,127,cases


In [3]:
df.describe()

C:\Users\RAVIT\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,time_period,time_period_type
count,0.0,0.0
mean,NaN,NaN
std,NaN,NaN
min,NaN,NaN
25%,NaN,NaN
50%,NaN,NaN
75%,NaN,NaN
max,NaN,NaN


In [4]:
df.columns.tolist()                                  #converting columns names to list

['location',
 'location_type',
 'data_field',
 'data_field_code',
 'time_period',
 'time_period_type',
 'value',
 'unit']

In [5]:
df['country'] = df['location']
df['country'] = df['country'].astype(str)
df['country'] = df['country'].apply(lambda x: pd.Series(x.split('-')))
df.country.unique()                                                                   #  all infected countries      

array(['Argentina', 'Brazil', 'Norte', 'Nordeste', 'Sudeste', 'Sul',
       'Centro', 'Colombia', 'nan', 'Dominican_Republic', 'Ecuador',
       'El_Salvador', 'Guatemala', 'Haiti', 'Mexico', 'Nicaragua',
       'Panama', 'Puerto_Rico', 'United_States',
       'United_States_Virgin_Islands'], dtype=object)

In [6]:
mask_brazil = df['country'] == "Brazil"
brazil_frame = df[mask_brazil]
brazil_frame.country.unique()

array(['Brazil'], dtype=object)

In [7]:
# Brazil  reports locations at the state level
brazil_frame = brazil_frame.drop(['location_type', 'data_field_code', 'time_period_type', 'time_period', 'unit'], axis=1)

In [8]:
## Creating state/city column 
foo = lambda x: pd.Series([i for i in reversed(x.split('-'))])
brazil_frame['location'] = brazil_frame.location.apply(foo)
brazil_frame.rename(columns={'location':'state_city'}, inplace=True)

In [9]:
## Cleaning the state city column
brazil_frame.state_city = brazil_frame.state_city.map(lambda x: x.replace('_',' '))
brazil_frame.state_city.value_counts()
                            # city wise data

Bahia                  171
Ceara                  171
Piaui                  171
Pernambuco             171
Rio Grande do Norte    171
Paraiba                171
Goias                  171
Alagoas                171
Parana                 170
Maranhao               170
Sergipe                170
Minas Gerais           170
Sao Paulo              170
Acre                   170
Mato Grosso            170
Rio Grande do Sul      162
Distrito Federal       161
Espirito Santo         159
Rio de Janeiro         159
Tocantins              158
Amapa                  143
Roraima                134
Santa Catarina         131
Rondonia               128
Para                   126
Mato Grosso do Sul     114
Amazonas               113
Brazil                   7
Name: state_city, dtype: int64

In [10]:
country_df = df[df['location_type']=='country']

all_countries = set(country_df['location'])

In [11]:
data_categories = sorted(country_df['data_field'].unique())
print(os.linesep.join(data_categories))

Zika_confirmed_F
Zika_confirmed_M
confirmed_age_1-4
confirmed_age_10-14
confirmed_age_15-19
confirmed_age_20-24
confirmed_age_25-34
confirmed_age_35-49
confirmed_age_5-9
confirmed_age_50-59
confirmed_age_60-64
confirmed_age_60_plus
confirmed_age_under_1
cumulative_confirmed
cumulative_suspected_age_1-4
cumulative_suspected_age_10-19
cumulative_suspected_age_20-29
cumulative_suspected_age_30-39
cumulative_suspected_age_40-49
cumulative_suspected_age_5-9
cumulative_suspected_age_50-59
cumulative_suspected_age_60_plus
cumulative_suspected_age_under_1
cumulative_suspected_pregnant
cumulative_suspected_total
efe_reported
gbs_confirmed_cumulative
gbs_reported
gbs_reported_4weeks
gbs_reported_cumulative
microcephaly_confirmed_cumulative
microcephaly_suspected
microcephaly_suspected_4weeks
microcephaly_suspected_cumulative
normal_birth_confirmed_zika
total_zika_confirmed
total_zika_confirmed_ages_0-11mo_F
total_zika_confirmed_ages_0-11mo_M
total_zika_confirmed_ages_1-4yrs_F
total_zika_confirme

In [12]:

# Clustring the data in terms of age groups 


age_groups = ('confirmed_age_under_1', 'confirmed_age_1-4',
       'confirmed_age_5-9', 'confirmed_age_10-14', 'confirmed_age_15-19',
       'confirmed_age_20-24', 'confirmed_age_25-34', 'confirmed_age_35-49',
       'confirmed_age_50-59', 'confirmed_age_60-64',
       'confirmed_age_60_plus')

for i,age_group in enumerate(age_groups):
    print (age_group)
    print (df[df.data_field==age_group].value)
    print ("")

confirmed_age_under_1
report_date
2016-04-11    6
Name: value, dtype: object

confirmed_age_1-4
report_date
2016-04-11    10
Name: value, dtype: object

confirmed_age_5-9
report_date
2016-04-11    8
Name: value, dtype: object

confirmed_age_10-14
report_date
2016-04-11    15
Name: value, dtype: object

confirmed_age_15-19
report_date
2016-04-11    6
Name: value, dtype: object

confirmed_age_20-24
report_date
2016-04-11    18
Name: value, dtype: object

confirmed_age_25-34
report_date
2016-04-11    38
Name: value, dtype: object

confirmed_age_35-49
report_date
2016-04-11    50
Name: value, dtype: object

confirmed_age_50-59
report_date
2016-04-11    20
Name: value, dtype: object

confirmed_age_60-64
report_date
2016-04-11    7
Name: value, dtype: object

confirmed_age_60_plus
report_date
2016-04-11    4
Name: value, dtype: object



In [13]:
# distribution on the basis of location ( can be anything like , country, state , city ) 

df.groupby("location").size().reset_index().rename(columns={0: "count"})


,location,count
0,Argentina-Buenos_Aires,84
1,Argentina-CABA,84
2,Argentina-Catamarca,84
3,Argentina-Chaco,84
4,Argentina-Chubut,84
5,Argentina-Cordoba,84
6,Argentina-Corrientes,84
7,Argentina-Entre_Rios,84
8,Argentina-Formosa,84
9,Argentina-Jujuy,84


In [14]:

## analysis of brazil  country as we know that it is higly infected with the ZICA virus 



brazil_frame = brazil_frame[brazil_frame.state_city != 'Brazil']
def order(frame,var):
    varlist =[w for w in frame.columns if w not in var]
    frame = frame[var+varlist]
    return frame

brazil_frame = order(brazil_frame,['report_date','country', 'state_city'])
## Keeping microcephaly_confirmed and microcephaly_fatal_confirmed
mask_conf = (brazil_frame['data_field'] == "microcephaly_confirmed") | (brazil_frame['data_field'] == "microcephaly_fatal_confirmed")
brazil_frame = brazil_frame[mask_conf]
def int_replace(x):
    try:
        return int(x)
    except:
        return np.nan_to_num(x)

brazil_frame.value = brazil_frame.value.map(int_replace)
brazil_frame.value = brazil_frame.value.astype(int)
## Reshaping the Data
brazil_zika = pd.pivot_table(brazil_frame,
                             index=['country','state_city','report_date'],
                             columns=['data_field'],values=['value'],
                             aggfunc=sum)
brazil_zika = brazil_zika['value'].reset_index()
brazil_zika.sort_values("report_date", inplace=True)
brazil_zika.set_index("report_date", inplace=True)
brazil_zika.index = brazil_zika.index.to_datetime()
grouped = brazil_zika.groupby(['month'])

grouped_val = grouped['microcephaly_confirmed']
category_group=grouped_val.sum()

grouped_val_1 = grouped['microcephaly_fatal_confirmed']
category_group_1=grouped_val_1.sum()


plt.figure(figsize=(10,6))

braz_graph= category_group.plot(kind='bar', color='green', alpha=0.7, 
                                title= "Microcephaly Cases 2016\n", 
                                label="microcephaly_confirmed")
braz_graph_1= category_group_1.plot(kind='bar', color='red', alpha=0.7)

braz_graph.set_xlabel("Months\n")
braz_graph.set_ylabel("Total Cases\n")

plt.tick_params(labelsize=14)
plt.legend(loc='upper left')
plt.show()

KeyError: "['report_date'] not in index"